# Function Examples
In this notebook we show some examples of the problem solvers provided by Ingenii, based on the [Azure Quantum package](https://pypi.org/project/azure-quantum/).

### Let's get your credentials to connect to the workspace we'll submit jobs to.

First, you'll need to establish your connection to the workspace using the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/what-is-azure-cli).

If you're not in our JupyterLab space, you'll need to [install the CLI first](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli).

Then, in a terminal, run the `az login` command to take you through the authentication flow. If you don't do this now, you will be prompted later on.

### As a first example, let's look at allocating weights between containers

In [ ]:
from ingenii_azure_quantum.functions import AllocateWeightsBetweenContainers

# All Ingenii algorithms have a several attributes, which tell you what the algorithm does and how to use it
print("Decription:", AllocateWeightsBetweenContainers.description)
print("\n")
print("Required Parameters:", AllocateWeightsBetweenContainers.required_parameters)
print("\n")
print("Optional Parameters:", AllocateWeightsBetweenContainers.optional_parameters)
print("\n")
print("Example Parameters:", AllocateWeightsBetweenContainers.example_parameters)

#### The algorithm class contains details of what it requires to run

#### As an example, let's initialise the algorithm using the default parameters

In [ ]:
example_allocation = AllocateWeightsBetweenContainers(**AllocateWeightsBetweenContainers.example_parameters)

#### As a first go, let's solve this using default settings
If you're in our JupyterLab space then the workspace details are pre-configured in environment variables.

If you're not in our JupyterLab workspace, either:
 - set the WORKSPACE_SUBSCRIPTION_ID, WORKSPACE_RESOURCE_GROUP, WORKSPACE_LOCATION, and WORKSPACE_NAME environment variables
 - pass these values directly to the get_workspace function, as below
 - A mixture of the two

```
example_allocation.set_workspace(
    subscription_id="<subscription ID>",
    resource_group="<resource group name>",
    location="<location, e.g. EastUS>",
    name="<workspace name>",
)
```

If you did not run `az login` before, the below step will now prompt you

In [ ]:
example_allocation.run()
example_allocation.visualize_allocation_result()

#### This is more adavnced, but from the results we can extract the beta_start, beta_stop, restarts, and sweeps parameters that the solution found.
#### Then, we can call the solver again using these parameters, to find the solution more quickly.

In [ ]:
# Update the 'solver_parameters' dictionary:
result = example_allocation.result
example_allocation.solver_parameters = {
    "beta_start": result["parameters"]["beta_start"],
    "beta_stop": result["parameters"]["beta_stop"],
    "restarts": result["parameters"]["restarts"],
    "sweeps": result["parameters"]["sweeps"],
}

# Re-run the problem with the same object to overwrite the results
example_allocation.run()
example_allocation.visualize_allocation_result()

#### Here's how we could experiment with different solvers from Microsoft
It's possible to update and re-use the same object, but below we create new objects for each approach

In [ ]:
from azure.quantum.optimization import HardwarePlatform, ParallelTempering, \
    QuantumMonteCarlo, SimulatedAnnealing, Tabu

example_parameters = AllocateWeightsBetweenContainers.example_parameters

print("#### Solution with platform FPGA")
example_allocation_2 = AllocateWeightsBetweenContainers(
    **example_parameters, solver=SimulatedAnnealing,
    timeout=5, platform=HardwarePlatform.FPGA
)
example_allocation_2.run()
example_allocation_2.visualize_allocation_result()

print("\n")
print("#### Solution using Tabu solver")
example_allocation_3 = AllocateWeightsBetweenContainers(
    **example_parameters, solver=Tabu, timeout=5
)
example_allocation_3.run()
example_allocation_3.visualize_allocation_result()

print("\n")
print("#### Solution using ParallelTempering solver")
example_allocation_4 = AllocateWeightsBetweenContainers(
    **example_parameters, solver=ParallelTempering, timeout=60
)
example_allocation_4.run()
example_allocation_4.visualize_allocation_result()

print("\n")
print("#### Solution using QuantumMonteCarlo solver")
example_allocation_5 = AllocateWeightsBetweenContainers(
    **example_parameters, solver=QuantumMonteCarlo
)
example_allocation_5.run()
example_allocation_5.visualize_allocation_result()


#### Here's how we can submit the same jobs to solvers by third-party providers, such as 1QBit
Note: PathRelinkingSolver is only available if the 1QBit provider is enabled in your quantum workspace

In [ ]:
from azure.quantum.target.oneqbit import PathRelinkingSolver

print("#### Solution using 1QBit PathRelinkingSolver solver")
example_allocation_1qbit = AllocateWeightsBetweenContainers(
    **AllocateWeightsBetweenContainers.example_parameters,
    solver=PathRelinkingSolver
)
example_allocation_1qbit.run()
example_allocation_1qbit.visualize_allocation_result()

### As a second example, we can allocate jobs between machines to complete them as quickly as possible
The outline of this problem is:
 - There are several jobs that need to be completed
 - Each job is made up of several individual operations that need to be completed in order
 - Each operation needs to be run on a specific machine
 - A machine can only execute one operation at a time
 - We want to find the best way to allocate these operations

In [ ]:
from ingenii_azure_quantum.functions import ScheduleJobsBetweenMachines

print("Decription:", ScheduleJobsBetweenMachines.description)
print("\n")
print("Required Parameters:", ScheduleJobsBetweenMachines.required_parameters)
print("\n")
print("Optional Parameters:", ScheduleJobsBetweenMachines.optional_parameters)
print("\n")
print("Example Parameters:", ScheduleJobsBetweenMachines.example_parameters)

#### Set the problem parameters

In [ ]:
## Allowed time (jobs can only be scheduled below this limit)
max_time = 21 

## Processing time for each operation
ops_processing_time = {0: 2, 1: 1, 2: 3, 3: 2, 4: 2, 5: 3, 6: 1, 7: 2, 8: 3, 9: 2}

## Assignment of operations to jobs (job ID: [operation IDs])
### Operation IDs within a job must be in ascending order
jobs_ops_map = {
    0: [0, 1, 2],   # Pay electricity bill
    1: [3, 4, 5],   # Plan camping trip
    2: [6, 7, 8, 9] # Book dentist appointment
}

## Assignment of operations to machines
### Ten jobs, three machines
machines_ops_map = {
    0: [0, 1, 3, 4, 6, 7],  # Operations 0, 1, 3, 4, 6 and 7 are assigned to machine 0 (the computer)
    1: [2, 5, 8],           # Operations 2, 5 and 8 are assigned to machine 1 (the printer)
    2: [9]                  # Operation 9 is assigned to machine 2 (the tooth floss)
}


#### Let's submit this job and see the results

In [ ]:
schedule_jobs_example = ScheduleJobsBetweenMachines(
    jobs_ops_map=jobs_ops_map,
    ops_processing_time=ops_processing_time,
    machines_ops_map=machines_ops_map,
    max_time=max_time, timeout=100
)
schedule_jobs_example.run()

result = schedule_jobs_example.result

print(f"Valid result: {result['valid']}")
print(f"Operation start times: {result['operation_start_times']}")
print(f"Job start times: {result['job_start_times']}")

#### We can visualise this result to see more clearly what's going on
By default, our workspace requires more packages to be installed for this cell to work.
In a terminal, run the below:
```
pip install plotly
pip install pandas
pip install nbformat
```

In [ ]:
import plotly.express as px

nb_ops = len(ops_processing_time)

# Graphics
ops_to_machines = [None] * nb_ops
for m, ops in machines_ops_map.items():
    for op in ops:
        ops_to_machines[op] = m

ops = sorted(list(ops_processing_time.keys()))

op_start_times = [
    result["operation_start_times"][op_id]
    for op_id in range(nb_ops)
]
op_end_times = [
    start_time + ops_processing_time[i]
    for i, start_time in enumerate(op_start_times)
]

# Create data frame
df = {
    "Operation": ops,
    "Machine": ops_to_machines,
    "Start time": op_start_times,
    "Finish time": op_end_times,
    "delta": [ops_processing_time[i] for i in range(nb_ops)],
}

# Produce plot
fig = px.timeline(df, title="Job Shop Schedule", color="Machine",
                  x_start="Start time", x_end="Finish time", y="Operation")

fig.update_yaxes(autorange="reversed")
fig.update_layout(
    font_family="Segoe UI",
    title_font_family="Segoe UI",
    width=800,
    height=800,
)
fig.layout.xaxis.type = "linear"
fig.data[0].x = df["delta"]
fig.update_layout(
    xaxis_title="Time step",
)
fig.update_yaxes(tick0=0, dtick=1)
fig.update_xaxes(tick0=0, dtick=1)
fig.show()

### Finally, let's look at a solution to the [Travelling Salesman problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem)
> This is here as an example: the Traveling Salesperson Problem is not a good example of a problem that scales well in the format that it is derived, as detailed in [this paper](https://arxiv.org/abs/1702.06248).

In [ ]:
from ingenii_azure_quantum.functions import TravelingSalesman

print("Decription:", TravelingSalesman.description)
print("\n")
print("Required Parameters:", TravelingSalesman.required_parameters)
print("\n")
print("Optional Parameters:", TravelingSalesman.optional_parameters)
print("\n")
print("Example Parameters:", TravelingSalesman.example_parameters)

#### Let's define the cost to travel between nodes 

In [ ]:
# Note this matrix is not symmetric (traveling A->B is not same as B->A!)
cost_matrix = [
    [1, 4, 7, 4, 3],
    [3, 3, 3, 1, 2],
    [2, 5, 2, 3, 1],
    [7, 8, 1, 3, 5],
    [3, 2, 1, 9, 8]
]
# If you want to run with a randomised cost matrix, uncomment the below
# import numpy as np
# nb_nodes = 5
# max_cost = 10
# cost_matrix = np.random.randint(maxCost, size=(nb_nodes, nb_nodes))

#### Let's see the result using the default approach

In [ ]:
# approach_type=SimulatedAnnealing
traveling_salesman_example = TravelingSalesman(cost_matrix=cost_matrix)   
traveling_salesman_example.run()

print(traveling_salesman_example.result)

#### There are different approaches that can be used, returning varying quality of results at different speeds

In [ ]:
from azure.quantum.optimization import ParallelTempering, QuantumMonteCarlo, Tabu

print("#### Solution using ParallelTempering solver")
traveling_salesman_example_2 = TravelingSalesman(cost_matrix=cost_matrix, solver=ParallelTempering, timeout=120)   
traveling_salesman_example_2.run()

print(traveling_salesman_example_2.result)

print("\n")
print("#### Solution using Tabu solver")
traveling_salesman_example_3 = TravelingSalesman(cost_matrix=cost_matrix, solver=Tabu, timeout=120)   
traveling_salesman_example_3.run()

print(traveling_salesman_example_3.result)

print("\n")
print("#### Solution using QuantumMonteCarlo solver")
traveling_salesman_example_4 = TravelingSalesman(cost_matrix=cost_matrix, solver=QuantumMonteCarlo, sweeps=2, trotter_number=10, restarts=72, seed=22, beta_start=0.1, transverse_field_start=10, transverse_field_stop=0.1) # QMC is not available parameter-free yet)   
traveling_salesman_example_4.run()

print(traveling_salesman_example_4.result)